# Модуль 06 - Объединение датафреймов
## Блок 02 - Метод merge

In [23]:
import pandas as pd
import numpy as np

genres = ['Drama', 'Action', 'Thriller']
ratings = pd.read_csv('module_05_ratings.csv')
movies = pd.read_csv('module_06_movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# Считаем количество фильмов в жанре Fantasy.
movies['genres'].str.contains('Fantasy').sum()
# Сначала использовал count() балда... :) на первых порах всегда нужно узнать через info() 
# все параметры dataframe

654

In [24]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [10]:
# проверка на отсутствие дубликатов после объединения dataframe's
len(ratings) == len(joined)

True

In [17]:
# Даны выгрузки статистики рекламных кампаний из Яндекс.Директа (файл “direct_stats.tsv”) 
# и заказов на сайте из CRM (файл “crm_stats.tsv”).

# В качестве разделителя используем \t
direct_stats = pd.read_csv('module_06_direct_stats.tsv', '\t')
crm_stats = pd.read_csv('module_06_crm_stats.tsv', '\t')
# Необходимо совместить эти выгрузки по дате и названию кампании. И посчитать стоимость 
# заказа для каждой кампании за каждый день. Т. е. отношение расходов (столбец cost) 
# к количеству заказов (столбец orders)

In [18]:
direct_stats.head()

,date,campaign,views,clicks,cost
0,2018-01-01,landings_promo,38120423,49557,1139801
1,2018-01-01,homepage_partner_1,5729483,12605,189073
2,2018-01-01,homepage_partner_2,4412029,9265,176040
3,2018-01-01,socdem_w_25-34_vip_test,913823,2559,89555
4,2018-01-02,landings_promo,40873806,61311,1471457


In [19]:
crm_stats.head()

,date,campaign,orders
0,2018-01-01,landings_promo,1487
1,2018-01-01,homepage_partner_1,386
2,2018-01-01,homepage_partner_2,315
3,2018-01-01,socdem_w_25-34_vip_test,85
4,2018-01-02,landings_promo,1605


In [21]:
direct_crm_joined = direct_stats.merge(crm_stats, on=['date','campaign'], how='left')
direct_crm_joined.head()

,date,campaign,views,clicks,cost,orders
0,2018-01-01,landings_promo,38120423,49557,1139801,1487
1,2018-01-01,homepage_partner_1,5729483,12605,189073,386
2,2018-01-01,homepage_partner_2,4412029,9265,176040,315
3,2018-01-01,socdem_w_25-34_vip_test,913823,2559,89555,85
4,2018-01-02,landings_promo,40873806,61311,1471457,1605


In [23]:
# проверка на дубликаты
len(direct_stats) == len(direct_crm_joined)

True

In [33]:
direct_crm_joined['order_cost'] = direct_crm_joined['cost'] / direct_crm_joined['orders']
direct_crm_joined.head()

,date,campaign,views,clicks,cost,orders,price,order_cost
0,2018-01-01,landings_promo,38120423,49557,1139801,1487,766.510424,766.510424
1,2018-01-01,homepage_partner_1,5729483,12605,189073,386,489.826425,489.826425
2,2018-01-01,homepage_partner_2,4412029,9265,176040,315,558.857143,558.857143
3,2018-01-01,socdem_w_25-34_vip_test,913823,2559,89555,85,1053.588235,1053.588235
4,2018-01-02,landings_promo,40873806,61311,1471457,1605,916.795639,916.795639


In [48]:
# Какое значение стоимости заказа получится к кампании landings_promo за 1 января 2018?
# Результат округлить до целого.
order_cost = direct_crm_joined[(direct_crm_joined['campaign'] == 'landings_promo') 
                   & (direct_crm_joined['date'] == '2018-01-01')]['order_cost'].sum()
# Использовал sum() для преобразования результата из dataframe в float64. полагаю это не лучший вариант.
round(order_cost)

767

766.5104236718224

In [50]:
# Возвращаемся к dataframe "joined"
# Напишем отдельную функцию для этой операции, а потом применим ее к каждой строке датафрейма joined 
# с помощью метода apply. Соответственно, логика функции должна быть такой:
# 1. Функция должна возвращать столько же столбцов, сколько элементов в списке genres.
# 2. Вместо названий жанров надо подставлять значение столбца rating, 
#    если жанр есть в столбце genres датафрейма.
# 3. Если названия жанра не оказалось, то возвращаем np.NaN

# используем list comprehension для умещения функции в одну строку
def genres_ratings(row):
    return pd.Series([row['rating'] if genre in row['genres'] else np.NAN for genre in genres])


In [62]:
# Применяем функцию к dataframe

joined[genres] = joined.apply(genres_ratings, axis = 1)
joined.head()

,userId,movieId,rating,timestamp,title,genres,Drama,Action,Thriller
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,2.5,NaN,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,3.0,NaN,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN,NaN,3.0
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,NaN,2.0,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,4.0,NaN,NaN


In [54]:
# Если жанр встречается в текущей строке, то присвоить ему значение столбца rating. 
# Если жанра в строке нет, то ставим пустое значение np.NaN (т. е. указываем, что в ячейке нет числа).
#
# Здесь нам пригодится библиотека numpy (которую мы обозначили как np). 
# А пустое значение ставим, чтобы при подсчете среднего эти строки не учитывались в результате.
# Можно было ставить 0, но тогда это сильно исказит итоговый средний рейтинг жанра.

for genre in genres:
    print('Жанр {} средний рейтинг {:.2f}'.format(genre, joined[genre].mean()))

Жанр Drama средний рейтинг 3.68
Жанр Action средний рейтинг 3.45
Жанр Thriller средний рейтинг 3.52


In [25]:
# Необходимо установить, какие жанры фильмов имеют меньше всего оценок в наших данных.
# В качестве списка жанров используйте расширенную версию:
genres_extended = ['Drama', 'Action', 'Thriller', 'Comedy', 'Romance', 'War', 'Mystery', 'Crime']

def genres_ratings_extended(row):
    return pd.Series([row['rating'] if genre in row['genres'] 
                      else np.NAN for genre in genres_extended])
joined_extended = ratings.merge(movies, on='movieId', how='left')
joined_extended[genres_extended] = joined.apply(genres_ratings_extended, axis = 1)
joined_extended.head()

,userId,movieId,rating,timestamp,title,genres,Drama,Action,Thriller,Comedy,Romance,War,Mystery,Crime
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
joined_extended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100004 entries, 0 to 100003
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
 4   title      100004 non-null  object 
 5   genres     100004 non-null  object 
 6   Drama      44752 non-null   float64
 7   Action     27056 non-null   float64
 8   Thriller   25240 non-null   float64
 9   Comedy     38026 non-null   float64
 10  Romance    19336 non-null   float64
 11  War        5025 non-null    float64
 12  Mystery    7625 non-null    float64
 13  Crime      16266 non-null   float64
dtypes: float64(9), int64(3), object(2)
memory usage: 11.4+ MB


In [27]:
# Какое количество оценок имеет жанр с наименьшим количеством рейтингов?

# Вычисляем количество оценок по всем жанрам
joined_extended[genres_extended].count()

Drama       44752
Action      27056
Thriller    25240
Comedy      38026
Romance     19336
War          5025
Mystery      7625
Crime       16266
dtype: int64

In [30]:
# получаем жанр с минимальным количеством оценок. Можно было сделать и на предыдущем шаге, 
# но для наглядности сделал так.
joined_extended[genres_extended].count().min()

5025

## Блок 03 - Подводные камни объединения датафреймов

In [1]:
import pandas as pd
ratings = pd.read_csv('module_06_ratings_example.txt', '\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [2]:
movies = pd.read_csv('module_06_movies_example.txt', '\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [3]:
ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [5]:
# Удаляем дубликаты из dataframe по movieId
movies.drop_duplicates(subset='movieId', keep='first', inplace=True)
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [6]:
# Обновляю dataframe movie для задания самопроверки
movies = pd.read_csv('module_06_movies_example.txt', '\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [17]:
print('-=- left -=-')
print(ratings.merge(movies, how='left', on='movieId'))
print('-=- right -=-')
print(ratings.merge(movies, how='right', on='movieId'))
print('-=- inner -=-')
print(ratings.merge(movies, how='inner', on='movieId'))
print('-=- outer -=-')
print(ratings.merge(movies, how='outer', on='movieId'))

-=- left -=-
   userId  movieId  rating   timestamp                    title genres
0       1       31     2.5  1260759144  Dangerous Minds (1995)   Drama
1       1       31     2.5  1260759144  Dangerous Minds (1995)   Drama
-=- right -=-
   userId  movieId  rating     timestamp  \
0     1.0       31     2.5  1.260759e+09   
1     NaN       32     NaN           NaN   
2     1.0       31     2.5  1.260759e+09   

                                       title                   genres  
0                    Dangerous Minds (1995)                     Drama  
1  Twelve Monkeys (a.k.a. 12 Monkeys) (1995)  Mystery|Sci-Fi|Thriller  
2                    Dangerous Minds (1995)                     Drama  
-=- inner -=-
   userId  movieId  rating   timestamp                    title genres
0       1       31     2.5  1260759144  Dangerous Minds (1995)   Drama
1       1       31     2.5  1260759144  Dangerous Minds (1995)   Drama
-=- outer -=-
   userId  movieId  rating     timestamp  \
0     1.0 

In [21]:
# Одинаковые результаты в случае left и outer, в случае right и outer итоговый dataframe 
# имеет одинаковое наполнение, но разный порядок следования строк.

## Блок 04 - Домашнее задание

In [22]:
# Есть мнение, что «раньше снимали настоящее кино, не то что сейчас». Ваша задача проверить это утверждение,
# используя файлы с рейтингами фильмов из материалов занятия, т. е. проверить верно ли, что с ростом года 
# выпуска фильма его средний рейтинг становится ниже.
#
# При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, 
# а пройдемся по следующему алгоритму:
#
# 1. В переменную years запишите список из всех годов с 1950 по 2010.
#
# 2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. 
#    Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте 
#    следующий алгоритм:
#
#    для каждой строки пройдите по всем годам списка years;
#    если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска;
#    если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год;
#    учтите, что год в списке years является числом, поэтому для проверки его наличия в заголовке фильма 
#    надо преобразовать его в строку с помощью str (year).
#
# 3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'.
#
# 4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат 
#    по убыванию рейтинга. Напоминание: для таких задач проще всего использовать метод groupby.
#
# Для размышления: верно ли, что в итоговой таблице 'старые' фильмы 1950-60-х годов в основном находятся 
# в верхней части таблицы, а в конце таблицы более новые фильмы?
#
# В каком году выпускались фильмы с максимальным средним рейтингом? Ответ укажите в виде целого числа 
# года выпуска.

In [88]:
import pandas as pd

ratings = pd.read_csv('module_05_ratings.csv')
movies = pd.read_csv('module_06_movies.csv')

In [89]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [90]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [91]:
# Создаём список годов выпуска фильмов с 1950 по 2010 с шагом 1 и сразу преобразуем в str
years = [str(y) for y in range(1950, 2011)]
print (years)

['1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']


In [92]:
# Объединяю dataframes ratings и movies
joined = ratings.merge(movies, on='movieId', how='left')
joined

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
...,...,...,...,...,...,...
99999,671,6268,2.5,1065579370,Raising Victor Vargas (2002),Comedy|Drama|Romance
100000,671,6269,4.0,1065149201,Stevie (2002),Documentary
100001,671,6365,4.0,1070940363,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX
100002,671,6385,2.5,1070979663,Whale Rider (2002),Drama


In [93]:
len(ratings) == len (joined)

True

In [94]:
# Функция определения года выпуска фильма 
def production_year(row):
    for year in years:
        if year in row['title']:
            return year
    return '1900'

In [95]:
# Определяем год выпуска фильма, если он не в диапазоне 1950 - 2010, то год указан 1900
joined['year'] = joined.apply(production_year, axis = 1)
joined.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


In [115]:
# Создаём новый dataframe только с необходимыми столбцами
result = joined.drop(['userId', 'movieId', 'timestamp', 'title', 'genres'], axis = 1)
result

,rating,year
0,2.5,1995
1,3.0,1900
2,3.0,1996
3,2.0,1981
4,4.0,1989
...,...,...
99999,2.5,2002
100000,4.0,2002
100001,4.0,2003
100002,2.5,2002


In [116]:
# Десятка лидеров рейтинга
result.groupby('year').mean().sort_values('rating', ascending=False).head(10)

,rating
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.994220
1951,3.983539
1974,3.978704
1962,3.952446
1950,3.915254
1977,3.905786


In [117]:
# Десятка аутсайдеров рейтинга
result.groupby('year').mean().sort_values('rating', ascending=False).tail(10)

,rating
year,
1999,3.471692
1992,3.466754
2001,3.460052
2002,3.456297
1990,3.449169
2005,3.448434
2003,3.444777
1996,3.426600
1997,3.415764


In [ ]:
# Максимальный рейтинг у фильмов 1957, 1972, 1952, 1954 годов